In [0]:
import os
from pyspark.sql.functions import col, expr, substring, split, regexp_replace, when
from pyspark.sql.functions import sum, round
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, lpad, lit
import pandas as pd

In [0]:
spark.catalog.clearCache()

In [0]:
df2020 = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/uhii_label_data_2020.csv")
)
df2021 = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/uhii_label_data_2021.csv")
)
df2022 = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/uhii_label_data_2022.csv")
)
df2023 = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/uhii_label_data_2023.csv")
)
df2024 = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/uhii_label_data_2024.csv")
)

df_sdot = df2020.union(df2021).union(df2022).union(df2023).union(df2024)

In [0]:
df_sdot = df_sdot.withColumn("year", substring(col("time_window"), 1, 4))
df_sdot = df_sdot.withColumn("month", substring(col("time_window"), 7, 1))
df_sdot = df_sdot.withColumn("day", substring(col("time_window"), 9, 2))
df_sdot = df_sdot.withColumn("hour", substring(col("time_window"), 12, 2))
display(df_sdot.limit(1))

In [0]:
df_join = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/join_v1.csv/")
)
df_sgg = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/project/default/project/sigungu-code.csv")
)

In [0]:
df_sdot.groupBy("year").count().orderBy("year").show()
df_sdot.printSchema()

In [0]:
df_sdot = df_sdot.join(df_sgg, df_sdot.URBAN_DISTRICT == df_sgg.name_kor).drop(
    "name_kor", "code", "URBAN_DISTRICT", "time_window"
)
display(df_sdot.limit(1))

In [0]:
display(df_sdot.filter(df_sdot["name_eng"]=="Seongbuk-gu").filter(df_sdot["month"] == "7").limit(1))

In [0]:
display(df_join.filter(df_join["month"]==7).limit(1))

In [0]:
df_join_cleaned = df_join.withColumn("year_str", col("year").cast("string"))
df_join_cleaned = df_join_cleaned.withColumn("month_str", col("month").cast("string"))
df_join_cleaned = df_join_cleaned.drop("year", "month")

In [0]:
joined_df = df_join_cleaned.join(
    df_sdot,
    (df_join_cleaned.District == df_sdot.name_eng)
    & (df_join_cleaned.year_str == df_sdot.year)
    & (df_join_cleaned.month_str == df_sdot.month),
).drop("name_eng", "year_str", "month_str")

In [0]:
display(joined_df)

In [0]:
joined_df.groupBy("year").count().orderBy("year").show()

In [0]:
df=joined_df.drop("driving_total_km","road_km")

In [0]:
output_path = "/Volumes/project/default/project/join_v2.csv"
df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)

print(f"DataFrame이 '{output_path}' 경로에 성공적으로 덮어쓰여졌습니다.")